In [ ]:
Prompt System YAML Détaillé pour Système Agentique Obstétrical


In [ ]:
# Configuration du système agentique d'intelligence obstétricale et santé fœtale
apiVersion: obstetrics.ai/v1
kind: AgentOrchestratorConfig
metadata:
  name: obstetric-intelligence-system
spec:
  mode: run_all
  execution_mode: sequential
  agents:
    - id: CTGMonitorAgent
      name: "CTG Monitor Agent"
      description: "Surveillance CTG (FHR, STV/LTV) et classification FIGO du rythme fœtal"
      inputs:
        - "Données CTG en temps réel (fréquence cardiaque fœtale)"
      outputs:
        narrative_summary: "Résumé narratif (~300 mots) de l'état du rythme cardiaque fœtal et conclusions (normal/suspect/pathologique selon FIGO)."
        critical_metrics:
          - metric: "Fréquence cardiaque de base"
            value: "<calculée>"
            unit: "bpm"
            confidence_interval: "±5 bpm"
            provenance: "Capteur CTG"
          - metric: "Variabilité STV"
            value: "<calculée>"
            unit: "ms"
            confidence_interval: "±1 ms"
            provenance: "Analyse CTG"
          - metric: "Classification FIGO"
            value: "<Normal/Suspect/Pathologique>"
            unit: "categorie"
            confidence_interval: null
            provenance: "Guide FIGO 2015"
        situation_report: "Situation normale, aucune escalade requise."
      llm_settings:
        temperature: 0.2
        max_tokens: 512
        top_p: 0.9
        presence_penalty: 0.0
        prompt_template: "CTGAnalysisPrompt"
      fhir_output: "Observation (analyse CTG)"
      rest_endpoint:
        path: "/api/ctg-monitor"
        method: POST
        input_schema: "FHIR Observation (CTG brut)"
        output_schema: "FHIR Observation (résultats FHR/STV/LTV)"
    - id: BishopPartogramAgent
      name: "Bishop & Partogram Agent"
      description: "Évaluation du score de Bishop et suivi du partogramme pour le travail"
      inputs:
        - "Examens cliniques (dilatation, effacement, consistance du col)"
        - "Historique du travail (partogramme)"
      outputs:
        narrative_summary: "Résumé narratif (~300 mots) sur l'état du col (score de Bishop) et progression du travail (partogramme)."
        critical_metrics:
          - metric: "Score de Bishop"
            value: "<calculé>"
            unit: "/13"
            confidence_interval: null
            provenance: "Examen clinique"
          - metric: "Phase du travail"
            value: "<Latente/Active>"
            unit: "categorie"
            confidence_interval: null
            provenance: "Partogramme"
        situation_report: "Travail en cours (phase active), pas d'anomalie, continuer suivi."
      llm_settings:
        temperature: 0.3
        max_tokens: 400
        top_p: 0.9
        presence_penalty: 0.0
        prompt_template: "BishopPartogramPrompt"
      fhir_output: "Observation (Score de Bishop & progression travail)"
      rest_endpoint:
        path: "/api/bishop-partogram"
        method: POST
        input_schema: "FHIR Observation (examen col utérin, données partogramme)"
        output_schema: "FHIR Observation (résultats score Bishop/progression)"
    - id: RCIURiskAgent
      name: "RCIU Risk Agent"
      description: "Évaluation du risque de RCIU (Retard de Croissance Intra-Utérin) à partir des données fœtales"
      inputs:
        - "Données biométriques fœtales (échographies, poids estimé)"
        - "Facteurs de risque maternels (antécédents, nutrition)"
      outputs:
        narrative_summary: "Résumé (~300 mots) du risque de RCIU, incluant les percentiles de croissance fœtale et recommandations."
        critical_metrics:
          - metric: "Risque de RCIU"
            value: "<calculé>"
            unit: "%"
            confidence_interval: "IC95%"
            provenance: "Courbe de croissance fœtale"
          - metric: "Poids fœtal estimé"
            value: "<calculé>"
            unit: "grammes"
            confidence_interval: "±200g"
            provenance: "Échographie 32 SA"
        situation_report: "Risque RCIU faible, surveillance standard maintenue."
      llm_settings:
        temperature: 0.25
        max_tokens: 500
        top_p: 0.9
        presence_penalty: 0.0
        prompt_template: "RCIURiskPrompt"
      fhir_output: "RiskAssessment (RCIU)"
      rest_endpoint:
        path: "/api/rciu-risk"
        method: POST
        input_schema: "FHIR Observation/RiskAssessment (données de croissance)"
        output_schema: "FHIR RiskAssessment (risque RCIU %)"
    - id: ApgarTransitionAgent
      name: "Apgar & Postnatal Transition Agent"
      description: "Évaluation du score d'Apgar du nouveau-né et suivi de l'adaptation néonatale"
      inputs:
        - "Observation clinique du nouveau-né (Apgar à 1 et 5 minutes)"
        - "Signes vitaux postnataux (FC, respiration, tonus)"
      outputs:
        narrative_summary: "Résumé (~300 mots) de l'état du nouveau-né (scores Apgar, interventions requises, adaptation postnatale)."
        critical_metrics:
          - metric: "Score Apgar 1 min"
            value: "<calculé>"
            unit: "/10"
            confidence_interval: null
            provenance: "Évaluation clinique"
          - metric: "Score Apgar 5 min"
            value: "<calculé>"
            unit: "/10"
            confidence_interval: null
            provenance: "Évaluation clinique"
        situation_report: "Apgar 5 min = 7, surveillance néonatale renforcée, pédiatre informé."
      llm_settings:
        temperature: 0.2
        max_tokens: 300
        top_p: 0.9
        presence_penalty: 0.0
        prompt_template: "ApgarEvaluationPrompt"
      fhir_output: "Observation (Score Apgar)"
      rest_endpoint:
        path: "/api/apgar-transition"
        method: POST
        input_schema: "FHIR Observation (scores Apgar, signes vitaux nouveau-né)"
        output_schema: "FHIR Observation (résultats Apgar & transition)"
    - id: SymbolicReasoningAgent
      name: "Symbolic Reasoning Agent"
      description: "Vérification symbolique des recommandations (guidelines HAS, FIGO, CNGOF) vis-à-vis des données du cas"
      inputs:
        - "Base de connaissances (protocoles HAS, FIGO, CNGOF)"
        - "Synthèse des données patient (sorties des autres agents)"
      outputs:
        narrative_summary: "Analyse (~300 mots) de conformité du cas aux recommandations officielles (points d'alignement ou écarts)."
        critical_metrics:
          - metric: "Conformité aux protocoles HAS"
            value: "<Oui/Non>"
            unit: "booléen"
            confidence_interval: null
            provenance: "Règles HAS 2022"
          - metric: "Écarts identifiés"
            value: "<calculé>"
            unit: "nombre"
            confidence_interval: null
            provenance: "Comparaison FIGO/CNGOF"
        situation_report: "Aucun écart majeur détecté par rapport aux guides cliniques."
      llm_settings:
        temperature: 0.1
        max_tokens: 600
        top_p: 0.8
        presence_penalty: 0.0
        prompt_template: "GuidelineCompliancePrompt"
      fhir_output: "DetectedIssue (écarts aux recommandations)"
      rest_endpoint:
        path: "/api/symbolic-reasoning"
        method: POST
        input_schema: "FHIR Bundle (données patient consolidées, connaissances)"
        output_schema: "FHIR DetectedIssue/Observation (écarts aux recommandations)"
    - id: PolygraphVerifierAgent
      name: "Polygraph & Hallucination Verifier Agent"
      description: "Vérification croisée des sorties des agents (détection d'hallucinations ou incohérences)"
      inputs:
        - "Sorties narratives des agents LLM"
        - "Base de connaissances médicale vérifiée"
      outputs:
        narrative_summary: "Rapport (~300 mots) de vérification indiquant la fiabilité des conclusions (points vérifiés, doutes ou hallucinations détectés)."
        critical_metrics:
          - metric: "Indice de confiance global"
            value: "<calculé>"
            unit: "score 0-1"
            confidence_interval: null
            provenance: "Analyse croisée LLM vs connaissances"
          - metric: "Risque d'hallucination"
            value: "<calculé>"
            unit: "score 0-1"
            confidence_interval: null
            provenance: "Détection Polygraph"
        situation_report: "Confiance >= 0.95 sur toutes les sorties, pas d'alerte d'hallucination."
      llm_settings:
        temperature: 0.0
        max_tokens: 400
        top_p: 1.0
        presence_penalty: 0.0
        prompt_template: "TruthVerifierPrompt"
      fhir_output: "Observation (qualité de sortie IA)"
      rest_endpoint:
        path: "/api/polygraph-verify"
        method: POST
        input_schema: "JSON (rapports des autres agents)"
        output_schema: "FHIR Observation (scores de confiance/hallucination)"
    - id: QuantumBirthOptimizerAgent
      name: "Quantum Birth Optimizer Agent"
      description: "Calcul du moment optimal pour déclencher l'accouchement (optimisation quantique/Monte Carlo)"
      inputs:
        - "État actuel mère et fœtus (dilatation, CTG, risques)"
        - "Scénarios de déclenchement possibles"
      outputs:
        narrative_summary: "Rapport (~300 mots) sur le scénario optimal d'accouchement (moment idéal du déclenchement, justification et probabilité de succès)."
        critical_metrics:
          - metric: "Probabilité de naissance sans complication"
            value: "<calculée>"
            unit: "%"
            confidence_interval: "IC90%"
            provenance: "Simulation quantique"
          - metric: "Heure optimale de déclenchement"
            value: "<calculée>"
            unit: "heures"
            confidence_interval: "±1h"
            provenance: "Algorithme d'optimisation"
        situation_report: "Recommandation: déclenchement dans 2 heures (probabilité succès 85%)."
      llm_settings:
        temperature: 0.3
        max_tokens: 500
        top_p: 0.9
        presence_penalty: 0.0
        prompt_template: "BirthOptimizationPrompt"
      fhir_output: "Observation (recommandation déclenchement)"
      rest_endpoint:
        path: "/api/quantum-optimizer"
        method: POST
        input_schema: "FHIR Bundle (données mère-bébé actuelles)"
        output_schema: "FHIR Observation (recommandation timing optimal)"
    - id: MotherBabyRiskAgent
      name: "Mother-Baby Risk Correlation Agent"
      description: "Analyse conjointe mère/enfant (corrélations de risques via Monte Carlo / SHAP)"
      inputs:
        - "Facteurs de risque maternels (âge, antécédents, conditions)"
        - "Facteurs de risque fœtaux (RCIU, anomalies CTG)"
      outputs:
        narrative_summary: "Synthèse (~300 mots) des risques combinés mère-enfant, avec corrélations identifiées et impact sur le pronostic."
        critical_metrics:
          - metric: "Corrélation risque mère-enfant"
            value: "<calculée>"
            unit: "coefficient r"
            confidence_interval: "IC95%"
            provenance: "Analyse Monte Carlo"
          - metric: "Indice de risque combiné"
            value: "<calculé>"
            unit: "score"
            confidence_interval: "±0.1"
            provenance: "Modèle SHAP"
        situation_report: "Risque obstétrical global modéré, pas d'intervention additionnelle nécessaire."
      llm_settings:
        temperature: 0.25
        max_tokens: 450
        top_p: 0.9
        presence_penalty: 0.0
        prompt_template: "MotherBabyRiskPrompt"
      fhir_output: "RiskAssessment (risque combiné mère-enfant)"
      rest_endpoint:
        path: "/api/mother-baby-risk"
        method: POST
        input_schema: "FHIR Bundle (facteurs risques mère et fœtus)"
        output_schema: "FHIR RiskAssessment (risque corrélé)"
    - id: ClinicalNarrativeAgent
      name: "Clinical Narrative Generator Agent"
      description: "Génération d'un rapport clinique consolidé (narratif) avec sources à destination des cliniciens"
      inputs:
        - "Ensemble des conclusions des autres agents (données agrégées)"
        - "Références et justifications (guidelines utilisées)"
      outputs:
        narrative_summary: "Rapport clinique global (~300 mots) intégrant les résultats de tous les agents, avec citations des sources et recommandations finales."
        critical_metrics:
          - metric: "Sources citées"
            value: "<calculé>"
            unit: "nombre"
            confidence_interval: null
            provenance: "Guidelines et données utilisées"
          - metric: "Lisibilité (score Flesch)"
            value: "<calculé>"
            unit: "score"
            confidence_interval: null
            provenance: "Analyse du texte généré"
        situation_report: "Rapport final généré pour le clinicien (prêt à être validé et partagé)."
      llm_settings:
        temperature: 0.4
        max_tokens: 1000
        top_p: 0.95
        presence_penalty: 0.0
        prompt_template: "ClinicalSummaryPrompt"
      fhir_output: "Composition (rapport clinique)"
      rest_endpoint:
        path: "/api/clinical-narrative"
        method: POST
        input_schema: "FHIR Bundle (toutes données agents & sources)"
        output_schema: "FHIR Composition (rapport clinique complet)"
    - id: UserEngagementAgent
      name: "User Engagement Agent"
      description: "Interaction avec le clinicien et la famille (explications adaptées, questions/réponses)"
      inputs:
        - "Préférences utilisateur (langue, niveau de détail)"
        - "Rapport clinique et données clés pour explication"
      outputs:
        narrative_summary: "Résumé (~300 mots) ou messages adaptés pour le patient/la famille, expliquant la situation en termes simples."
        critical_metrics:
          - metric: "Score de satisfaction"
            value: "<calculé>"
            unit: "/5"
            confidence_interval: null
            provenance: "Feedback utilisateur"
          - metric: "Nombre d'interactions"
            value: "<calculé>"
            unit: "count"
            confidence_interval: null
            provenance: "Session d'engagement"
        situation_report: "Informations fournies à la famille, compréhension confirmée, pas de questions en suspens."
      llm_settings:
        temperature: 0.7
        max_tokens: 500
        top_p: 0.95
        presence_penalty: 0.3
        prompt_template: "UserEngagementPrompt"
      fhir_output: "Communication (compte-rendu patient/famille)"
      rest_endpoint:
        path: "/api/user-engagement"
        method: POST
        input_schema: "FHIR Bundle (rapport clinique + préférences utilisateur)"
        output_schema: "FHIR Communication (messages au patient/famille)"
  orchestration:
    fallback_strategy: "Utilisation de valeurs par défaut sûres en cas d'échec d'un agent, poursuite du pipeline."
    audit_log: true
    audit_log_hash: "SHA256"
    human_in_the_loop:
      triggers:
        - "CTGMonitorAgent: FIGO Pathologique"
        - "ApgarTransitionAgent: Apgar5min <= 6"
      action: "pause_and_notify_human"
    security_threshold: 0.1
    on_security_breach: "halt_and_flag"
  final_output:
    global_summary: "Synthèse globale finale du cas (avec citations des guidelines et données sources)."
    aggregated_metrics:
      outcomes_by_agent:
        CTGMonitorAgent: "CTG Normal"
        BishopPartogramAgent: "Phase active, Bishop=8"
        RCIURiskAgent: "Faible risque RCIU"
        ApgarTransitionAgent: "Apgar 5min = 7/10"
        SymbolicReasoningAgent: "Conforme aux protocoles"
        PolygraphVerifierAgent: "Sorties cohérentes"
        QuantumBirthOptimizerAgent: "Déclenchement optimisé dans 2h"
        MotherBabyRiskAgent: "Risque global modéré"
        ClinicalNarrativeAgent: "Rapport clinique généré"
        UserEngagementAgent: "Explication patient fournie"
      risk_flags:
        - "Apgar faible (détresse modérée)"
        - "Surveillance fœtale accrue recommandée"
      recommendations:
        - "Surveillance rapprochée du travail"
        - "Préparer équipe néonatale par précaution"
    polygraph_surveillance:
      confidence_threshold: 0.90
      cross_verification_method: "Vérification croisée symbolique vs LLM"
      result: "Confiance globale >= 0.95, aucune incohérence majeure."
  compliance:
    EU_AI_Act_Annex_IV: true
    ISO_14971: true
    HIPAA: true
    traceability: "Traçabilité complète des artefacts (prompts, sorties, décisions)"
    logs_integrity: "Hashes SHA256 appliqués aux logs"
